In [1]:
import ASW
import mat_properties as prop
import pandas as pd


RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
accumulation = pd.read_excel("blocks.xlsx", sheet_name="accumulation", index_col=0)
ASWatm = True
ASWbul = 2
vremya = 4
vremyaojidaniya=12

In [2]:
import numpy as np
from numpy import linalg as LA
# ----------------------------- Начало Максима Опарина
ASW = ASW.Accum(water,water_streams,accumulation,ASWatm,
                     stream12 = "ASW-WOUT",
                     stream11 = "SP2-ASW",
                     stream_obratnoi_setevoi_vody = "SWIN-TURB",
                     stream_pryamoi_setevoi_vody = "SP2-WOUT",
#                      T_nar_vozd = water_streams.at["AIR", "T"]
                    T_nar_vozd = -25
               )

ASW.set_construct(Diametr=6,
                       kolichestvo = 2,
                       Visota = 10,
                       lambda_min_vata = 0.045,
                       delta_min_vata = 0.01)

if ASWatm == False:
    
    
    # # если Зарядка   
    if ASWbul == 1:
        G_ASW_zarydka = ASW.zaryadka(vremya)['G']
        print(G_ASW_zarydka,'G')
        water_streams.at["SWIN-TURB", "G"] = water_streams.at["SWIN", "G"] + G_ASW_zarydka
        water_streams.at["SP2-WOUT", "G"] = water_streams.at["SWIN", "G"] + G_ASW_zarydka
            ## если разрядка
    if ASWbul == 2:
        ASW.zaryadka(vremya)
        ASW.jdat_n(vremyaojidaniya)
        G_ASW_razryadka = ASW.razryadka(vremya)['G']
        print(G_ASW_razryadka,'check')
        water_streams.at["SWIN-TURB", "G"] = water_streams.at["SWIN", "G"] - G_ASW_razryadka
        water_streams.at["SP2-WOUT", "G"] = water_streams.at["SWIN", "G"] - G_ASW_razryadka
        water_streams.at["SWOUT", "H"] = water.p_t(water_streams.at["SWOUT","P"],water_streams.at["SWOUT", "T"])['h']
        print(water_streams.at["SWOUT", "P"], 'Check P')
        print(water_streams.at["SWOUT", "T"], 'Check T')  
        print(water_streams.at["SWOUT", "H"],'check H')
        water_streams.at["SP2-WOUT", "H"] = (water_streams.at["SWOUT", "H"]*water_streams.at["SWOUT", "G"] -
                                            water_streams.at["ASW-WOUT", "H"]*G_ASW_razryadka)/(water_streams.at["SWOUT", "G"]-G_ASW_razryadka)
        water_streams.at["SP2-WOUT", "T"] = water.p_h(water_streams.at["SP2-WOUT", "P"],water_streams.at["SP2-WOUT", "H"])['T']
else:

    # # если Зарядка   
    if ASWbul == 1:
        G_ASW_zarydka = ASW.zaryadka(vremya)['G']
        print(G_ASW_zarydka,'GATM')
        
        a = [[1, 1],
        [1*water_streams.at["SP2-WOUT", "H"], 1*water_streams.at["SWIN-TURB", "H"]]]
#         b = [G_ASW_zarydka, G_ASW_zarydka*water.p_t(0.1,95)['h']]
        b = [G_ASW_zarydka, G_ASW_zarydka*water.p_t(0.1,min(95,water_streams.at["SP2-WOUT", "T"]))['h']]
        print(water_streams.at["SP2-WOUT", "H"],water_streams.at["SWIN-TURB", "H"])
        print(G_ASW_zarydka,water.p_t(0.1,95)['h'])
        x = LA.solve(a, b)
        G1 = x[0]
        G2 = x[1]
        print(x)
        water_streams.at["SWIN-ASWatm", "G"] = G2 = x[1]
        
        water_streams.at["SWIN-TURB", "G"] = water_streams.at["SWIN", "G"] + G_ASW_zarydka - G2 
        water_streams.at["SP2-WOUT", "G"] = water_streams.at["SWIN", "G"] + G_ASW_zarydka - G2
            ## если разрядка
    if ASWbul == 2:
        ASW.zaryadka(vremya)
        ASW.jdat_n(vremyaojidaniya)
        G_ASW_razryadka = ASW.razryadka(vremya)['G']
        print(G_ASW_razryadka,'check')
        water_streams.at["SWIN-TURB", "G"] = water_streams.at["SWIN", "G"] - G_ASW_razryadka
        water_streams.at["SP2-WOUT", "G"] = water_streams.at["SWIN", "G"] - G_ASW_razryadka
        water_streams.at["SWOUT", "H"] = water.p_t(water_streams.at["SWOUT","P"],water_streams.at["SWOUT", "T"])['h']
        water_streams.at["SP2-WOUT", "H"] = (water_streams.at["SWOUT", "H"]*water_streams.at["SWOUT", "G"] -
                                            water_streams.at["ASW-WOUT", "H"]*G_ASW_razryadka)/(water_streams.at["SWOUT", "G"]-G_ASW_razryadka)
        water_streams.at["SP2-WOUT", "T"] = water.p_h(water_streams.at["SP2-WOUT", "P"],water_streams.at["SP2-WOUT", "H"])['T']
    
# ----------------------------- Конец Максима Опарина
print(accumulation)
print(water_streams)

377.06246187048646 h_accum
0.1 self._P_accum
90.0 self._T_accum
37.907595111722074 check
1.0 Check P
90.0 Check T
377.75901533255535 check H
      Qw           T        V
ASW  0.0   81.743953      NaN
PKM  NaN  250.000000  10000.0
                        T         P            H           G   S   X
AIR                  15.0       NaN          NaN         NaN NaN NaN
PEVD-DROSVD         511.5     8.407  3423.209978        63.4 NaN NaN
IVD-PEVD       301.297556     8.746  2747.104023        63.4 NaN NaN
EVD-IVD             298.8     8.746  1337.948265        63.4 NaN NaN
PEN-EVD             166.5     8.746   708.319849        63.4 NaN NaN
BND-PEN        164.738361    0.6964   696.095567        63.4 NaN NaN
PPND-DROSND         211.6     0.691  2871.481837       14.76 NaN NaN
IND-PPND       164.738361    0.6964   2762.53498       14.76 NaN NaN
GPK-IND             164.6    0.6964   695.493701       78.16 NaN NaN
GPK-REC             164.6    0.6964   695.493701       82.91 NaN NaN
REC-GPK   

In [3]:
# import mat_properties as prop
# gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# import numpy as np
# from numpy import linalg as LA

# RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
# water = prop.Materials_prop(
#     "water",
#     [1.0, 0, 0, 0, 0],
#     prop.REFPROP_h_s,
#     prop.REFPROP_p_t,
#     prop.REFPROP_p_h,
#     prop.REFPROP_p_s,
#     prop.REFPROP_p_q,
#     prop.REFPROP_t_q,
#     prop.REFPROP_p_rho,
#     prop.REFPROP_s_q,
#     RP=RP,
# )
# Tpr_sv = 120
# Tobr_sv=45
# T_ASW=95
# P_ASW=0.1
# P_pr_sv = 1

# G_ASW= 10

# hpr_sv = water.p_t(P_pr_sv, Tpr_sv)['h']
# print(hpr_sv,'hpr_sv')
# hobr_sv = water.p_t(P_pr_sv, Tobr_sv)['h']
# print(hobr_sv,'hobr_sv')
# h_ASW = water.p_t(P_ASW, T_ASW)['h']
# print(h_ASW,'h_ASW')

# a = [[1, 1],
#      [1*hpr_sv, 1*hobr_sv]]

# b = [G_ASW, G_ASW*h_ASW]

# x = LA.solve(a, b)
# G1 = x[0]
# G2 = x[1]
# print(G1,'vot oni otdelno',G2)

In [4]:
# ASW = ASW.Accum(water,water_streams,accumulation,ASWatm,
#                  stream12 = "ASW-WOUT",
#                  stream11 = "SP2-ASW", # при разряде
#                  stream_obratnoi_setevoi_vody = "SWIN-TURB",
#                  stream_pryamoi_setevoi_vody = "SP2-WOUT",
#                  T_nar_vozd = 15)

# ASW.set_construct(Diametr=2,
#                    kolichestvo = 2,
#                    Visota = 1,
#                    lambda_min_vata = 0.045,
#                    delta_min_vata = 0.01)

# ASW.zaryadka(1)
# ASW.jdat(4)
# ASW.razryadka(1)
# print(water_streams)
# print(heaters)